# Neel

In this notebook we will look at ferromagnetism by simulating the Hamiltonian

\begin{equation}
H = -\frac{1}{2} J \sum_{\langle i,j \rangle} S_{i}S_{j} - \mu_{B} B \sum_{i} S_{i},
\end{equation}

where $J$ is again the nearest neighbour exchange energy and $B$ is the external field. We will in first instance only take the exchange interaction between nearest neighbours into account and forget about the dipole term, we will choose $J$ to be negative.

In [ ]:
import numpy
import matplotlib.pyplot as plt

## The following package is required to update the plot within the same figure
from IPython.display import clear_output

To update our plots during the simulation we run in this notebook we define the function *live_plot* in the following cell. In the simulation, the left graph will show the spin everywhere in the lattice, on the top-right graph, the average magnetisation in the system is shown. In the bottom-right graph, you see the so-called staggered order parameter. It is negative when neighbouring spins are anti-aligned. 

In [ ]:
def live_plot(data_2D, data_M, data_MS):
    """
      Clear the figure and update the plots with newly created data
      
        :params:  - data_2D --> 2D numpy array
                  - data_1D --> Two 1D numpy arrays containing the x and y axis of the plotted data
    """
    clear_output(wait=True)
    
    ## Initialize the figure with corresponding subplots
    fig = plt.figure(figsize = (22, 10))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(2, 2, 2)
    ax3 = fig.add_subplot(2, 2, 4)
    
    ## Plot data_2D using imshow --> Direct visualization of the spin states in the system
    ax1.imshow(data_2D, interpolation = "none", vmin = -1., vmax = 1., cmap = "hot")

    ## Plot data_1D using plot --> Magnetization as a function of time. In this simulation time corresponds to simulation steps
    ax2.plot(data_M[:, 0], data_M[:, 1])
    
    ax2.set_ylim(-1.1, 1.1)
    ax2.set_ylabel("<S_i>")
    
    ax3.plot(data_MS[:, 0], data_MS[:, 1])
    
    ax3.set_ylim(-1.1, 1.1)
    ax3.set_xlabel("Monte Carlo Moves")
    ax3.set_ylabel("<-S_i*S_j>")
    
    plt.show();

Onwards to the actual notebook! First we define our system parameters by defining the number of particles in our system *N* and the number of _Steps_ the simulation is executed over.

In the Ising notebook we expressed everything in units of $\textit{J}$. Here, we will keep $\textit{J}$ explicitely because we want to be able to investigate what happens when $\textit{J} = 0$. Our paramaters are the neareast neighbour exhange energy $\textit{J}$, the external magnetic field $\textit{B}$, and the thermal energy $\textit{kBT}$.

In [ ]:
N = 50
Steps = 1000000

J = -2.0
B = -2.0
kBT = 3.0

## Calculate the energy change due to one spin flip
def deltaE(S,i,j):
    Sij=S[i,j]
    Enow=-Sij*(J*(S[(i+1)%N,j]+S[(i-1)%N,j]+S[i,(j+1)%N]+S[i,(j-1)%N])+B) # use periodic b.c.'s
    return -2*Enow

Now we initialize the system by randomly defining the spins. We do this by generating a two by two array with values between 0 and 1. All values below 0.5 correspond to spin down, *i.e.* -1 and all values above 0.5 correspond to spin up, _i.e._ +1.

In [ ]:
## Init the spins randomly (Remember: only down and up!)
spins = numpy.where(numpy.random.random((N, N)) > 0.5, 1, -1)

## Determine the magnitization and staggered order parameter. Store in numpy array with shape: (Steps, 2)
M     = numpy.zeros((Steps, 2))
MS    = numpy.zeros((Steps, 2))    ## Staggered order parameter
M[0]  = 0, numpy.array([numpy.mean(spins)])
MS[0] = 0, numpy.array([-numpy.mean(spins.flatten()[::2] * spins.flatten()[1::2])])


## Now define what we do for every step of the simulation
for idx in range(Steps):
    ## Select a random value for i and j
    i, j = numpy.random.randint(0, N, 2)
    ## For the selected indices i and j determine the change in energy
    dE = deltaE(spins, i, j)
    
    ## If the change in energy is smaller than 0, or a random number is smaller 
    ## than the corresponding thermal fluctuations, allow the spin change/flip
    if (dE < 0.0) or (numpy.random.random() < numpy.exp(-dE / float(kBT))):
        spins[i, j] = -spins[i, j]

    ## Determine the magnetization and the staggered order parameter of the system and store into the 
    M[idx]  = idx, numpy.mean(spins)
    MS[idx] = idx, numpy.mean(-numpy.mean(spins.flatten()[::2] * spins.flatten()[1::2]))
    
    ## For every 999th step, update the plots
    if idx %(999) == 0:       
        live_plot(spins, M[:idx], MS[:idx])